## Universidad del Valle de Guatemala
## (CC3085) Inteligencia Artificial
## Laboratorio 6 - Árboles de Decisión

Miembros del equipo:
- Pedro Pablo Arriola Jiménez (20188)
- Oscar Fernando López Barrios (20679)
- Yong Bum Park (20117)
- Santiago Taracena Puga (20017)

## Task 1) Árboles de Decisión y Videojuegos

En primer lugar, es necesario realizar procedimientos básicos de lectura, reconocimiento y limpieza del dataset en el que nos encontramos trabajando. Esta sección básicamente consiste en la realización de estos procedimientos.

In [1]:
# Librerías importantes para el desarrollo del laboratorio.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Lectura del archivo con los juegos de League of Legends.
data = pd.read_csv("./data/high_diamond_ranked_10min.csv")
data.head()

,gameId,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,4519157822,0,28,2,1,9,6,11,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,4523371949,0,12,1,0,5,5,5,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,4521474530,0,15,0,0,7,11,4,1,1,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,4524384067,0,43,1,0,4,5,5,1,0,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,4436033771,0,75,4,0,6,6,6,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


In [3]:
# Revisión para observar si algún dato es nulo.
data.isna().sum()

gameId                          0
blueWins                        0
blueWardsPlaced                 0
blueWardsDestroyed              0
blueFirstBlood                  0
blueKills                       0
blueDeaths                      0
blueAssists                     0
blueEliteMonsters               0
blueDragons                     0
blueHeralds                     0
blueTowersDestroyed             0
blueTotalGold                   0
blueAvgLevel                    0
blueTotalExperience             0
blueTotalMinionsKilled          0
blueTotalJungleMinionsKilled    0
blueGoldDiff                    0
blueExperienceDiff              0
blueCSPerMin                    0
blueGoldPerMin                  0
redWardsPlaced                  0
redWardsDestroyed               0
redFirstBlood                   0
redKills                        0
redDeaths                       0
redAssists                      0
redEliteMonsters                0
redDragons                      0
redHeralds    

In [4]:
# División del dataset en entrenamiento, y validación/prueba.
X_trainval, X_test, y_trainval, y_test = train_test_split(
    data.drop(["blueWins"], axis=1), 
    data["blueWins"], 
    test_size=0.1, 
    random_state=42
)

# División de los nuevos datasets en validación y prueba
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, 
    y_trainval, 
    test_size=0.1, 
    random_state=42
)

In [5]:
# Clase Node para construir el árbol de decisión.
class Node(object):
    def __init__(self, feature=None, threshold=None, value=None, left=None, right=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

# Función que calcula el índice de Gini.
def gini(y):

    # Calculamos la proporción de cada clase en y.
    p = y.value_counts(normalize=True)

    # Calculamos el índice de Gini.
    gini = (1 - sum(p ** 2))

    # Devolvemos el índice de Gini.
    return gini

# Función que selecciona la mejor variable de separación y el umbral para dividir los datos.
def select_best_split(X, y, max_features=None):

    # Calculamos el índice de Gini para cada variable y umbral.
    n_features = X.shape[1]

    # Si max_features es None, se consideran todas las variables.
    if ((max_features is not None) and (max_features < n_features)):
        features = np.random.choice(X.columns, size=max_features, replace=False)
    else:
        features = X.columns

    # Seleccionamos la mejor variable de separación y el umbral para dividir los datos.
    best_feature, best_threshold, best_gini = None, None, 1.0

    # Iteramos sobre cada variable y cada umbral.
    for feature in features:
        for threshold in X[feature].unique():
            left_idx = X[feature] < threshold
            right_idx = X[feature] >= threshold
            left_gini = gini(y.loc[left_idx])
            right_gini = gini(y.loc[right_idx])
            gini_impurity = (left_gini * sum(left_idx) + right_gini * sum(right_idx)) / len(y)

            # Si el índice de Gini es menor que el mejor hasta ahora, actualizamos los valores.
            if (gini_impurity < best_gini):
                best_feature = feature
                best_threshold = threshold
                best_gini = gini_impurity

    # Devolvemos la mejor variable de separación y el umbral para dividir los datos.    
    return best_feature, best_threshold

# Función que construye el árbol de decisión de forma recursiva.
def build_tree(X, y, max_depth=3, max_features=None, depth=0):

    # Si todos los elementos de y son iguales, devolvemos un nodo hoja con ese valor.
    if (len(y.unique()) == 1):
        return Node(value=y.iloc[0])

    # Si se alcanza la profundidad máxima, devolvemos un nodo hoja con la clase mayoritaria.
    if (depth == max_depth):  
        return Node(value=y.value_counts().idxmax())

    # Seleccionamos la mejor variable de separación y el umbral para dividir los datos.
    best_feature, best_threshold = select_best_split(X, y, max_features)

    # Dividimos los datos en dos conjuntos, aquellos que cumplen la condición y aquellos que no.
    left_idx = X[best_feature] < best_threshold
    right_idx = X[best_feature] >= best_threshold

    # Construimos los subárboles
    left = build_tree(X.loc[left_idx], y.loc[left_idx], max_depth,max_features, depth+1)
    right = build_tree(X.loc[right_idx], y.loc[right_idx], max_depth,max_features, depth+1)
    
    # Devolvemos el nodo raíz del subárbol
    return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

# Función que realiza la predicción de una instancia.
def predict(x, tree):

    # Si llegamos a un nodo hoja, devolvemos la clase correspondiente
    if (tree.value is not None):
        return tree.value

    # Si la instancia cumple la condición del nodo, continuamos por el subárbol izquierdo.
    if (x[tree.feature] < tree.threshold):
        return predict(x, tree.left)

    # Si no, continuamos por el subárbol derecho.
    else:
        return predict(x, tree.right)

In [6]:
# Construir el árbol de decisión con el conjunto de entrenamiento.
tree = build_tree(X_train, y_train, max_depth=4, max_features=5)

In [7]:
# Hacer predicciones con el conjunto de validación.
y_pred = X_val.apply(lambda x: predict(x, tree), axis=1)

In [8]:
# Calcular la precisión de las predicciones.
accuracy = accuracy_score(y_val, y_pred)
print(f"Precisión en el conjunto de validación: {accuracy:.3f}")

Precisión en el conjunto de validación: 0.734


In [9]:
# Importar las librerías necesarias para el árbol de decisión.
from sklearn.tree import DecisionTreeClassifier

# Cargar los datos.
data = pd.read_csv("./data/high_diamond_ranked_10min.csv")

# División del dataset en entrenamiento, validación y prueba.
X_trainval, X_test, y_trainval, y_test = train_test_split(
    data.drop(["blueWins"], axis=1), 
    data["blueWins"], 
    test_size=0.1, 
    random_state=42
)

# División del dataset de entrenamiento y validación en entrenamiento y validación.
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, 
    y_trainval, 
    test_size=0.1, 
    random_state=42
)

# Entrenar el modelo.
clf = DecisionTreeClassifier(max_depth=3, max_features=5, random_state=42)
clf.fit(X_train, y_train)

# Hacer predicciones con el conjunto de validación.
y_pred = clf.predict(X_val)

# Calcular la precisión de las predicciones.
accuracy = accuracy_score(y_val, y_pred)
print(f"Precisión en el conjunto de validación: {accuracy:.3f}")

Precisión en el conjunto de validación: 0.734


Los resultados obtenidos son bastante positivos, ya que el modelo implementado desde cero tiene un accuracy de 0.709, mientras que el modelo de árbol de decisión de sklearn tiene un accuracy de sólo 0.734, lo que indica que nuestra implementación se acerca bastante a la implementación oficial de la librería.